# Altered cards analisys

Set 1 il y a 81 (27 communes + 54 rares) cartes dans une faction. 3x pour compléter un set - soit 243 (plus héros / jetons)<br>
Set 2 

## import data with polars and build cards_df

In [ ]:
import json
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import re
import unidecode
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print(f'polars version: {pl.__version__}')
pl.Config.set_tbl_hide_column_data_types(True)

open json data as dict

In [ ]:
with open('results/cards.json', 'r', encoding='utf-8') as file:
    cards_dict = json.load(file)

In [ ]:
# Factions colors
factions_colors = {
    'AX': ('Axiom', '#704434'),
    'BR': ('Bravos', '#962f2a'),
    'LY': ('Lyra', '#eb658d'),
    'MU': ('Muna', '#507c49'),
    'OR': ('Ordis', '#066388'),
    'YZ': ('Yzmir', '#8143a2'),
}

show one card

In [ ]:
# cards_dict[list(cards_dict.keys())[50]]
for card_id, card_info in cards_dict.items():
    if card_info['name'].get('fr') == "Prestidigitatrice de l'Ouroboros" and card_info.get('mainFaction') == 'BR':
        if 'COREKS' in card_info['id']:
            pass
        break
card_info

In [ ]:
def clean_value(value):
    return int(value.replace('#', '')) if isinstance(value, str) else int(value)

def card_value(card_info, main_or_recall='main'):
    # card value
    ocean_power = clean_value(card_info['elements']['OCEAN_POWER'])
    mountain_power = clean_value(card_info['elements']['MOUNTAIN_POWER'])
    forest_power = clean_value(card_info['elements']['FOREST_POWER'])
    if main_or_recall == 'main':
        cost = clean_value(card_info['elements']['MAIN_COST'])
    else:
        cost = clean_value(card_info['elements']['RECALL_COST'])
    card_value = (ocean_power + mountain_power + forest_power)/3 / cost
    return card_value

In [ ]:
# Extract relevant details from each card in cards_dict
card_details = []
for card_id, card_info in cards_dict.items():
    if card_info['type'] == 'HERO' or card_info['type'] == 'TOKEN' or card_info['type'] == 'TOKEN_MANA' or card_info['type'] == 'FOILER' or 'COREKS' in card_info['id']:
        continue
    
    OCEAN_POWER = None
    MOUNTAIN_POWER = None
    FOREST_POWER = None
    main_value = None
    recall_value = None
    raw_value = None
    if card_info['type'] == 'CHARACTER':
        OCEAN_POWER = clean_value(card_info['elements'].get('OCEAN_POWER'))
        MOUNTAIN_POWER = clean_value(card_info['elements'].get('MOUNTAIN_POWER'))
        FOREST_POWER = clean_value(card_info['elements'].get('FOREST_POWER'))
        main_value = (OCEAN_POWER + MOUNTAIN_POWER + FOREST_POWER) / 3 / clean_value(card_info['elements'].get('MAIN_COST'))
        recall_value = (OCEAN_POWER + MOUNTAIN_POWER + FOREST_POWER) / 3 / clean_value(card_info['elements'].get('RECALL_COST'))
        raw_value = main_value + recall_value

    card_details.append({
        'id': card_info.get('id'),
        'type': card_info.get('type'),
        'mainFaction': card_info.get('mainFaction'),
        'rarity': card_info.get('rarity'),
        'name': card_info['name'].get('fr'),
        'Mcost': clean_value(card_info['elements'].get('MAIN_COST')),
        'Rcost': clean_value(card_info['elements'].get('RECALL_COST')),
        'Op': OCEAN_POWER,
        'Mp': MOUNTAIN_POWER,
        'Fp': FOREST_POWER,
        'MAIN_EFFECT': card_info['elements'].get('MAIN_EFFECT', {}).get('fr'),
        'ECHO_EFFECT': card_info['elements'].get('ECHO_EFFECT', {}).get('fr'),
        'main_value': main_value,
        'recall_value': recall_value,
        'raw_value': raw_value
    })

# Create a Polars DataFrame from the extracted details
cards_df = pl.DataFrame(card_details)
# Round the values to 1 decimal
cards_df = cards_df.with_columns([
    pl.col('main_value').round(1),
    pl.col('recall_value').round(1),
    pl.col('raw_value').round(1)
])
# Remove text in parentheses from the effects
cards_df = cards_df.with_columns([
    pl.col('MAIN_EFFECT').str.replace_all(r'\(.*?\)', '', literal=False).str.replace_all(r'#', '', literal=False).str.to_lowercase(),
    pl.col('ECHO_EFFECT').str.replace_all(r'\(.*?\)', '', literal=False).str.replace_all(r'#', '', literal=False).str.to_lowercase()
])
cards_df = cards_df.with_columns([
    pl.col('MAIN_EFFECT').str.replace_all(r'[\[\]]', '', literal=False),
    pl.col('ECHO_EFFECT').str.replace_all(r'[\[\]]', '', literal=False)
])

cards_df = cards_df.sort('type', 'raw_value', 'Mcost', descending=[False, True, False])
cards_df.head(3)

In [ ]:
def clean_print_df(cards_df, head=5):
    # Select columns to display, excluding 'id', 'type', and 'mainFaction'
    columns_to_display = [col for col in cards_df.columns if col not in ['id', 'type', 'mainFaction']]
    return cards_df.select(columns_to_display).head(head)

def plot_result_as_images(cards_df, ncol=5):
    nrow = math.ceil(len(cards_df) / ncol)
    fig, axes = plt.subplots(nrow, ncol, figsize=(15, 5 * nrow))
    axes = axes.flatten()
    
    for ax, row in zip(axes, cards_df.iter_rows(named=True)):
        card_img_path = f"assets\\cards_images\\{row['id']}.jpg"
        img = mpimg.imread(card_img_path)
        ax.imshow(img)
        ax.axis('off')
    
    for ax in axes[len(cards_df):]:
        ax.axis('off')
    
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.tight_layout()
    plt.show()

def plot_result_as_images_plotly(cards_df, ncol=5):
    nrow = math.ceil(len(cards_df) / ncol)
    fig = make_subplots(rows=nrow, cols=ncol, subplot_titles=[str(row['id']) for row in cards_df.iter_rows(named=True)])
    
    for i, row in enumerate(cards_df.iter_rows(named=True)):
        card_img_path = f"assets\\cards_images\\{row['id']}.jpg"
        img = mpimg.imread(card_img_path)
        
        fig.add_trace(
            go.Image(z=img),
            row=(i // ncol) + 1,
            col=(i % ncol) + 1
        )
    
    fig.update_layout(
        height=300 * nrow,
        width=300 * ncol,
        showlegend=False,
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    fig.show()


## Filter datas

### filter data

In [ ]:
# Filter the data by mainFaction, rarity, or name
filtered_df = cards_df.filter(
    (pl.col('mainFaction') == 'AX'),    # AX / BR / MU / OR / LY / YZ
    (pl.col('type') == 'CHARACTER'),    # CHARACTER / SPELL / BUILDING
    # (pl.col('MAIN_EFFECT').str.contains('{r}')),         # RARE / COMMON,
    
    # region h r j d
    # (pl.col('MAIN_EFFECT').str.contains(r'\{h\}'))      # from Hand effects
    # (pl.col('MAIN_EFFECT').str.contains(r'\{r\}'))      # from Reserve effects
    # (pl.col('ECHO_EFFECT').str.contains(r'\{d\}'))      # discard from reserve effects
    # endregion

    # region CONTROL EFFECTS / negative card advantage
    # (   # CONTROL EFFECTS / negative card advantage
    #     pl.col('MAIN_EFFECT').str.contains(r'défaussez (un|jusqu\'à deux cibles,) (personnage ou un permanent|personnage|permanent).*?(ciblé)?', literal=False) |
    #     pl.col('MAIN_EFFECT').str.contains(r'envoyez en réserve (un|jusqu\'à .*?) personnage(s)? ciblé(s)?', literal=False) |
    #     pl.col('MAIN_EFFECT').str.contains(r'sabotez', literal=False) |
    #     pl.col('MAIN_EFFECT').str.contains(r'(un personnage|chaque personnage dans une expédition) ciblé(e)? gagne (endormi|fugace)', literal=False) |
    #     pl.col('MAIN_EFFECT').str.contains(r'renvoyez un personnage ou un permanent ciblé', literal=False) |
    #     pl.col('MAIN_EFFECT').str.contains(r'gagnez (endormi|fugace)', literal=False) |
    #     pl.col('MAIN_EFFECT').str.contains('défausse ', literal=True)
    # ),
    # endregion

    # region BOOST
    # (   
    #     pl.col('MAIN_EFFECT').str.contains('boost')
    # ),
    # endregion

    # region # (   # CARD ADVANTAGE positive TODO: add encrière reserve to hand effect
    (   # CARD ADVANTAGE positive TODO: add encrière reserve to hand effect
        pl.col('MAIN_EFFECT').str.contains(r'gagne ancré', literal=False) |
        (pl.col('MAIN_EFFECT').str.contains(r'pioche(r|z) une carte') &
        ~pl.col('MAIN_EFFECT').str.contains(r'^(?:lorsqu).* pioche .*? carte(s)?')) |
        pl.col('MAIN_EFFECT').str.contains(r'ravitaille(r|z)') |
        pl.col('ECHO_EFFECT').str.contains(r'la prochaine carte que vous jouez ce tour') |
        pl.col('MAIN_EFFECT').str.contains(r'de votre réserve dans votre main') |
        pl.col('ECHO_EFFECT').str.contains(r'de votre réserve dans votre main')
    ),
    # endregion

    # region others
    # (pl.col('MAIN_EFFECT').str.contains(r'vous pouvez défausser un personnage ou un permanent ciblé', literal=False)),
    # endregion

)

#  filter
filtered_df = filtered_df.sort('mainFaction', 'Mcost', 'name', descending=[False, False, True])
print(f"Number of rows in filtered_df: {filtered_df.shape[0]}")
# Display the filtered data
clean_print_df(filtered_df)

### result of filters but with images directly

In [ ]:
# filtered_df = filtered_df.filter(pl.col('rarity') == 'RARE')
plot_result_as_images(filtered_df)

In [ ]:
unique_names = filtered_df.select('name').unique()
unique_names = unique_names.sort('name')
unique_names

In [ ]:
fig = go.Figure()

# Create a bar for each faction
for faction, (name, color) in factions_colors.items():
    faction_data = filtered_df.filter(pl.col('mainFaction') == faction)
    fig.add_trace(go.Bar(
        x=[name] * len(faction_data),
        y=[len(faction_data)],
        name=name,
        marker=dict(color=color),
        opacity=1
    ))

# Add annotations for each bar
annotations = []
for faction, (name, color) in factions_colors.items():
    faction_data = filtered_df.filter(pl.col('mainFaction') == faction)
    count = len(faction_data)
    if count > 0:
        annotations.append(dict(
            x=name,
            y=0,
            text=name,
            xref="x",
            yref="paper",
            showarrow=False,
            textangle=-0,
            font=dict(color='white', size=18, weight="bold"),
            xanchor='center',
            # yanchor='bottom'
        ))
        # Add annotations for mean Mcost and Rcost
    if len(faction_data) > 0:
        mean_mcost = faction_data['Mcost'].mean()
        mean_rcost = faction_data['Rcost'].mean()
        annotations.append(dict(
            x=name,
            y=0.18,
            xref="x",
            yref="paper",
            text=f'{mean_mcost:.1f}',   # f'{mean_mcost:.1f}',
            showarrow=False,
            font=dict(color='white', size=12),
            bgcolor='#5199c6',
            bordercolor='#5199c6',
            borderwidth=2,
            borderpad=4,
            opacity=1
        ))
        annotations.append(dict(
            x=name,
            y=0.09,
            xref="x",
            yref="paper",
            text=f'{mean_rcost:.1f}',
            showarrow=False,
            font=dict(color='white', size=12),
            bgcolor='#235175',
            bordercolor='#235175',
            borderwidth=2,
            borderpad=4,
            opacity=1
        ))
fig.update_layout(
    title='Histogram of boost cards by Faction',
    barcornerradius=15,
    xaxis=dict(
        tickmode='array',
        tickvals=[], # [name for name, color in factions_colors.values()],
        ticktext=[], # [name for name, color in factions_colors.values()],
        showticklabels=True  # Turn off x-axis ticks
    ),
    # xaxis_title='Factions',
    yaxis_title='Count',
    barmode='stack',
    bargap=0,  # Remove space between bars
    annotations=annotations,
    showlegend=False
)

fig.show()

In [ ]:
print(f"main value: {filtered_df['main_value'].mean():.1f}, recall value: {filtered_df['recall_value'].mean():.1f}, main mean raw value: {filtered_df['raw_value'].mean():.1f}")

## work on parsing phrases

show some phrases

In [ ]:
# Extract phrases from MAIN_EFFECT and ECHO_EFFECT columns
phrases_lst = []

# Iterate over the DataFrame rows
# for row in cards_df.iter_rows(named=True):
for row in filtered_df.iter_rows(named=True):
    ph = []
    main_effect_phrases = row['MAIN_EFFECT']
    # print(f'\n{main_effect_phrases}')
    if main_effect_phrases is not None:
        phs_split = re.split(r'\.\s\s| et ', main_effect_phrases)
        if phs_split is not None:
            for ph_split in phs_split:
                ph.append(ph_split)
        phrases_lst.append(ph)
    
    ph = []
    echo_effect_phrases = row['ECHO_EFFECT']
    if echo_effect_phrases is not None:
        phs_split = re.split(r'\.\s\s| et ', echo_effect_phrases)
        if phs_split is not None:
            for ph_split in phs_split:
                ph.append(ph_split)
        phrases_lst.append(ph)

final_phrases_lst = []
for phrases in phrases_lst:
    clean_phs = []
    for phrase in phrases:
        clean_phs.append(unidecode.unidecode(phrase))
    final_phrases_lst.append(clean_phs)

final_phrases_lst

In [ ]:
# Find and print the card with id "ALT_CORE_B_MU_16_R1"
card_id_to_find = "ALT_CORE_B_MU_16_R1"
if card_id_to_find in cards_dict:
    print(cards_dict[card_id_to_find])
else:
    print(f"Card with id {card_id_to_find} not found.")

In [ ]:
def parse_phrase(phrase):
    trigger_list = ['{j}', '{h}', '{r}', '{d}', '{t}', 'Lorsque']
    action_list = [
        (r'gagne(?:r)? (\d+) boost(?:s)?', 'pattern', 1, 'boost'),
        ('[sabotez]', 'classic', 1),         # +1 à value totale
        ('[ancre]', 'classic', 1),           # +1 fois les val des biomes moyennés
        (r'perdre \[\[fugace\]\]|perds \[\[fugace\]\]', 'pattern', 1, 'perds fugace'),   # -1 fois les val des biomes moyennés
        (r'gagne(?:r)? \[\[fugace\]\]', 'pattern', 1, 'gagne fugace'),   # -1 fois les val des biomes moyennés
        (r'coriace (\d+|x)', 'pattern', 0.2, 'coriace'),  # +0.3 à value totale
        ('aguerri', 'classic', 0.3),       # +1 fois les val des biomes moyennés 
        ('[gigantesque]', 'classic', 2),  # +2 fois les val des biomes moyennés
        ('pioche', 'classic', 1.5),           # +1 à value totale
        ('[ravitaillez]', 'classic', 1),       # +1 à value totale
    ]
    target_list = [
        'je ',
        'chaque joueur',
        'personnages qui me font face',
        'un autre personnage ciblé',
        'un personnage ciblé'
    ]
    condition_list = [r'(?:si|Si)(.*?)(?=[\[,\.])']
    
    for trigger in trigger_list:
        if trigger in phrase:
            break
        trigger = None
    
    print('\n', phrase)
    action = None
    add_value = None
    for act in action_list:
        if act[1] == 'pattern':
            match = re.findall(act[0], phrase)
            if match != []:
                action = act[3]
                try: add_value = int(match[0])
                except: add_value = act[2]
                break
        elif act[1] == 'classic':
            if act[0] in phrase:
                action = act[0]
                add_value = act[2]
                break

    # action_pattern = re.compile('|'.join(action_list))
    # condition_pattern = re.compile(condition_list[0])

    # triggers = trigger_pattern.findall(phrase)
    # actions = action_pattern.findall(phrase)
    # conditions = condition_pattern.findall(phrase)
    
    return {
        "trigger": trigger,
        "action": (action, add_value),
        # "conditions": conditions
    }

for phrases in phrases_lst:
    for phrase in phrases:
        print(parse_phrase(phrase))

In [ ]:
matches = re.findall(r'perdre \[\[Fugace\]\]|perds \[\[Fugace\]\]', '{J} Je gagne [[Fugace]].')
matches